#### Imports

In [1]:
# builtins
from time import time
# general
import numpy as np
import pandas as pd
import scipy
# statistical tests
from scipy.stats import ttest_ind, ranksums, ks_2samp
# spatial transcriptomics packages
import anndata as ad
import scanpy as sc
import squidpy as sq
# plotting
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import bokeh.io
import bokeh.plotting
import bokeh.layouts
import iqplot

bokeh.io.output_notebook()

Loading BokehJS ...

#### Loading in data

In [2]:
cell_assignments = pd.read_csv('cell_assignments_filtered.csv')
region_assignments = pd.read_csv('concordex_cerebellum_res_2024-06-10.csv')
cereb_coords = np.load('cerebellum_coords_mat.npy')
cereb_counts = np.load('cerebellum_counts_mat.npy')
gene_labels = np.load('cerebellum_gene_labels.npy', allow_pickle=True)

cell_assignments = cell_assignments['first_type']
region_assignments = region_assignments['concordex_pred']

df = pd.read_csv('df.csv')
comparisondf = pd.read_csv('comparisondf.csv')
passes_bonferroni = pd.read_csv('passes_bonferroni.csv')
marker_genes = pd.read_csv('celltype_marker_genes.csv')[['cell_type', 'gene']]

#### Preprocessing

In [3]:
# Setting up AnnData object
cereb_adata = ad.AnnData(cereb_counts)
cereb_adata.obs_names = ['Bead_'+str(i) for i in range(cereb_adata.n_obs)]
cereb_adata.var_names = gene_labels
cereb_adata.obs['cell_type'] = pd.Categorical(cell_assignments)
cereb_adata.obs['region'] = pd.Categorical(region_assignments)

# inverting y_coordinates because spatial plotting libraries were plotting the cerebellum
# upside down from usual display
x = cereb_coords[:, 0]
y = cereb_coords[:, 1]
new_y = y.max() - y
new_cereb_coords = np.array(list(zip(x, new_y)))
cereb_adata.obsm['spatial'] = new_cereb_coords


# Calculating % MT counts
mt_gene_ids = [i for i in range(14089, 14160)] + [i for i in range(23068, 23092)] # figured out these are the correct indices
mt_pcts = []
counts_matrix = cereb_adata.X
for bead_num in range(cereb_adata.n_obs):
    mt_pcts.append(np.sum(counts_matrix[bead_num][mt_gene_ids])/np.sum(counts_matrix[bead_num]))

cereb_adata.obs['pct_counts_mt'] = mt_pcts


# Filtering
print('Initial cells:', cereb_adata.n_obs)
print('Initial genes:', cereb_adata.n_vars)
sc.pp.filter_genes(cereb_adata, min_counts=50)
sc.pp.filter_genes(cereb_adata, min_cells=25)
cereb_adata = cereb_adata[cereb_adata.obs["pct_counts_mt"] < 0.1].copy()
print('Final cells:', cereb_adata.n_obs)
print('Final genes:', cereb_adata.n_vars)


# Finding total_counts (per gene) and n_cells_by_counts
cereb_adata.var.rename(columns={'n_counts':'total_counts', 'n_cells':'n_cells_by_counts'}, inplace=True)
    # filtering process somehow creates columns but we need to rename and reevaluate post-filtering

# Finding total_counts (gene) and n_cells_by_counts
cells_expressing  = []
expressions = []
X = cereb_adata.X.T 
    # transposes rows and columns so that genes are now rows
for gene_num in range(cereb_adata.n_vars):
    expressions.append(np.sum(X[gene_num]))
    cells_expressing.append(np.count_nonzero(X[gene_num]))

cereb_adata.var['n_cells_by_counts'] = cells_expressing
cereb_adata.var['total_counts'] = expressions


# Finding total_counts (cell) and n_genes_by_counts
genes_expressed  = []
total_counts = []
X = cereb_adata.X
for bead_num in range(cereb_adata.n_obs):
    total_counts.append(np.sum(X[bead_num]))
    genes_expressed.append(np.count_nonzero(X[bead_num]))

cereb_adata.obs['n_genes_by_counts'] = genes_expressed
cereb_adata.obs['total_counts'] = total_counts


# Normalization + log1p
sc.pp.normalize_total(cereb_adata)
sc.pp.log1p(cereb_adata)


# Finding post-normalization log1p_total_counts (cell and gene)
    # cell
total_counts = []
X = cereb_adata.X
for bead_num in range(cereb_adata.n_obs):
    total_counts.append(np.sum(X[bead_num]))

cereb_adata.obs['log1p_total_counts'] = total_counts

    # gene
total_counts = []
X = cereb_adata.X.T
for gene_num in range(cereb_adata.n_vars):
    total_counts.append(np.sum(X[gene_num]))

cereb_adata.var['log1p_total_counts'] = total_counts

Initial cells: 9985
Initial genes: 23096
Final cells: 9051
Final genes: 10293


### Various analysis

In [10]:
#                blue/gran   green/PB  orange/oligo red/MLI     gray
master_palette = ['#1A476F', '#55752F', '#E69F00', '#C10534','#C1C1C1']

#### General

In [ ]:
# individual stripbox + ECDF

gene_name = 'Lsamp'
cell_type = 'Astrocytes'

rows = passes_bonferroni[(passes_bonferroni['gene_name'] == gene_name) & 
                    (passes_bonferroni['cell_type'] == cell_type)]

if len(rows) != 0:
    regions = np.sort(pd.concat((rows['region1'], rows['region2'])).unique())
    handle = gene_name+'_'+cell_type
    
    # making df for plot
    cell_gene = cereb_adata[cereb_adata.obs['cell_type'] == cell_type][:, gene_name]
    
    ls1 = cell_gene.obs['region'] # regions
    ls2 = cell_gene.X.flatten()   # count data
    data = [[reg, ls2[ind]] for ind, reg in enumerate(ls1) if reg in regions] # only if it is of the regions we want
    index = list(range(len(ls1)))
    
    dfdict = dict(zip(index, data))
    col = ['region', 'log1p counts']
    newdf = pd.DataFrame.from_dict(data=dfdict, orient='index', columns=col)
    
    palette = [master_palette[i] for i in regions]
    
    box = iqplot.stripbox(
        data=newdf,
        q='log1p counts',
        cats='region',
        order=regions,
        frame_width=500,
        frame_height=300,
        spread='jitter',
        x_axis_label=gene_name+' counts in '+cell_type+' (log1p)',
        y_axis_label='region',
        palette=palette
    )

    # box.xaxis.axis_label_text_font_size = '12pt'
    # box.yaxis.axis_label_text_font_size = '12pt'
    box.xaxis.axis_label_text_font_style = 'normal'
    box.yaxis.axis_label_text_font_style = 'normal'
    
    ecdf = iqplot.ecdf(
        data=newdf,
        q='log1p counts',
        cats='region',
        order=regions,
        frame_width=500,
        frame_height=300,
        x_axis_label=gene_name+' counts in '+cell_type+' (log1p)',
        y_axis_label='cumulative probability',
        legend_location='bottom_right',
        palette=palette
    )

    # ecdf.xaxis.axis_label_text_font_size = '12pt'
    # ecdf.yaxis.axis_label_text_font_size = '12pt'
    ecdf.xaxis.axis_label_text_font_style = 'normal'
    ecdf.yaxis.axis_label_text_font_style = 'normal'
    
    multiplot = bokeh.layouts.gridplot([box, ecdf], ncols=1)

    # fname = './NewPassesBonferroni/'+handle+'_'+str(regions)+'.html'
    # bokeh.plotting.output_file(filename=fname)
    # bokeh.plotting.save(multiplot)

    bokeh.io.show(multiplot)

In [28]:
# individual gene: comparing regions regardless of cell type

gene_name = 'Fth1'
regions = [0, 1, 2, 3]

# making df for plot
gene = cereb_adata[:, gene_name]

ls1 = gene.obs['region'] # regions
ls2 = gene.X.flatten()   # count data
data = [[reg, ls2[ind]] for ind, reg in enumerate(ls1) if reg in regions] # only if it is of the regions we want
index = list(range(len(ls1)))

dfdict = dict(zip(index, data))
col = ['region', 'log1p counts']
newdf = pd.DataFrame.from_dict(data=dfdict, orient='index', columns=col)

palette = [master_palette[i] for i in regions]

box = iqplot.stripbox(
    data=newdf,
    q='log1p counts',
    cats='region',
    order=regions,
    frame_width=500,
    frame_height=300,
    spread='jitter',
    y_axis_label='region',
    x_axis_label=gene_name+' counts (log1p)',
    palette=palette
)

box.xaxis.axis_label_text_font_size = '12pt'
box.yaxis.axis_label_text_font_size = '12pt'
box.xaxis.axis_label_text_font_style = 'normal'
box.yaxis.axis_label_text_font_style = 'normal'
# box.x_range = bokeh.models.Range1d(-.25, 4.25)

ecdf = iqplot.ecdf(
    data=newdf,
    q='log1p counts',
    cats='region',
    order=regions,
    frame_width=500,
    frame_height=300,
    y_axis_label='cumulative probability',
    x_axis_label=gene_name+' counts (log1p)',
    legend_location='bottom_right',
    palette=palette
)

ecdf.xaxis.axis_label_text_font_size = '12pt'
ecdf.yaxis.axis_label_text_font_size = '12pt'
ecdf.xaxis.axis_label_text_font_style = 'normal'
ecdf.yaxis.axis_label_text_font_style = 'normal'
# ecdf.x_range = bokeh.models.Range1d(-.25, 4.25)

multiplot = bokeh.layouts.gridplot([box, ecdf], ncols=1)

# fname = './fth1_all_regions.html'
# bokeh.plotting.output_file(filename=fname)
# bokeh.plotting.save(multiplot)

bokeh.io.show(multiplot)

#### Checking for possible incorrect cell type assignments

In [6]:
# add in the code here

olis = cereb_adata[cereb_adata.obs['cell_type'] == 'Oligodendrocytes']
grans = cereb_adata[cereb_adata.obs['cell_type'] == 'Granule']
astros = cereb_adata[cereb_adata.obs['cell_type'] == 'Astrocytes']
purks = cereb_adata[cereb_adata.obs['cell_type'] == 'Purkinje']
mlis = cereb_adata[cereb_adata.obs['cell_type'] == 'MLI1']

reg0_olis = olis.obs_names[olis.obs['region'] == 0]
reg2_olis = olis.obs_names[olis.obs['region'] == 2]

reg0_grans = grans.obs_names[grans.obs['region'] == 0]

reg0_astros = astros.obs_names[astros.obs['region'] == 0]
reg1_astros = astros.obs_names[astros.obs['region'] == 1]

reg1_purks = purks.obs_names[purks.obs['region'] == 1]

reg1_mlis = mlis.obs_names[mlis.obs['region'] == 1]
reg3_mlis = mlis.obs_names[mlis.obs['region'] == 3]

In [9]:
# Test expression of marker genes of one cell type in a different cell type (by region)


for gene in marker_genes['gene'][marker_genes['cell_type'] == 'Granule']:
    if gene in cereb_adata.var.index:

        mli1_counts = np.array(cereb_adata[reg1_mlis][:, gene].X.flatten())
        mli3_counts = np.array(cereb_adata[reg3_mlis][:, gene].X.flatten())
        gran0_counts = np.array(cereb_adata[reg0_grans][:, gene].X.flatten())
        
        all_counts = np.concatenate((mli1_counts, mli3_counts, gran0_counts))
        celltype_names = ['MLI1 reg 1' for i in range(len(mli1_counts))] + ['MLI 1 reg 3' for i in range(len(mli3_counts))]+ ['Gran reg 0' for i in range(len(gran0_counts))]
        index = list(range(len(celltype_names)))
        
        dfdict = dict(zip(index, zip(all_counts, celltype_names)))
        col = ['counts', 'cell_type']
        newdf = pd.DataFrame.from_dict(dfdict, orient='index', columns=col)
        
        box = iqplot.stripbox(
            data=newdf,
            q='counts',
            cats='cell_type',
            frame_width=500,
            frame_height=300,
            spread='jitter',
        )
        
        ecdf = iqplot.ecdf(
            data=newdf,
            q='counts',
            cats='cell_type',
            frame_width=500,
            frame_height=300,
            legend_location='bottom_right'
        )
        
        multiplot = bokeh.layouts.gridplot([box, ecdf], ncols=1)

        fname = './IncorrectCellType/MLI1_GranMarkers/'+gene+'.html'
        bokeh.plotting.output_file(filename=fname)
        bokeh.plotting.save(multiplot)

#### MLI1 trend

In [29]:
# add in the code here

spatial_coords = np.array(cereb_adata.obsm['spatial'])

cereb_adata.obs['x_coord'] = spatial_coords[:, 0]
cereb_adata.obs['y_coord'] = spatial_coords[:, 1]

mli1 = cereb_adata.obs[cereb_adata.obs['cell_type'] == 'MLI1']
mli1_coords = np.array(list(zip(mli1['x_coord'], mli1['y_coord'])))

grans = cereb_adata.obs[cereb_adata.obs['cell_type'] == 'Granule']
gran_coords = np.array(list(zip(grans['x_coord'], grans['y_coord'])))

# create df of distances between every mli1 and granule
mli1_dists = scipy.spatial.distance.cdist(mli1_coords, gran_coords)
dists_df = pd.DataFrame(mli1_dists, index=mli1.index, columns=grans.index)

# calculate average distance to 10 nearest granules for each MLI1
avg_dists_mli1 = []
for key in dists_df.index:
    rowsort = dists_df.loc[key].sort_values()
    avg_dists_mli1.append(rowsort[:10].mean())
    
avg_dists_mli1 = np.array(avg_dists_mli1)



In [32]:
# definitely not the most efficient way to make this plot but it's what I came up with

goi1 = 'Aldoc'
far_goi1 = np.array(cereb_adata[far_inds][:, goi1].X).flatten()
close_goi1 = np.array(cereb_adata[close_inds][:, goi1].X).flatten()

goi2 = 'Sparcl1'
far_goi2 = np.array(cereb_adata[far_inds][:, goi2].X).flatten()
close_goi2 = np.array(cereb_adata[close_inds][:, goi2].X).flatten()


counts = np.concatenate((far_goi1, close_goi1, far_goi2, close_goi2))
dists = ['far' for i in range(len(far_goi1))] + ['close' for i in range(len(close_goi1))]
dists = dists*2
gene = [goi1 for i in range(len(mli1))] + [goi2 for i in range(len(mli1))] # + [goi3 for i in range(len(mli1))]

index = list(range(len(dists)))

col = ['gene', 'distance', 'counts']
dfdict = dict(zip(index, zip(gene, dists, counts)))

minidf = pd.DataFrame.from_dict(dfdict, columns=col, orient='index')


catplot_palette = ['#b49fd4', '#6d39bd']

sns.catplot(minidf, kind='violin', y='counts', 
            col='gene', hue='distance',
            split=True, inner='quart',
            palette=catplot_palette)

# plt.savefig('./mli1_activity.png', dpi=150, bbox_inches='tight')
plt.show()

NameError: name 'far_inds' is not defined